In [2]:
from brt.jit.tvm import TVMTuner
from brt.common import BRT_CACHE_PATH
from brt.common import log

log.set_level("jit", "DEBUG")

tuner = TVMTuner()
onnx_model = "sparse_fusion_2_thor_model"
tuner.import_onnx_netlet(onnx_model)
tuner.export_netlet_template()

Get devices for measurement successfully!
DEBUG:brainstorm.jit:kernel args: [[8, 64, 64], [8, 64, 64], [8, 64, 64]]


In [ ]:
from brt.jit import CUDACompiler
from brt.common import BRT_KERNEL_TEMPLATE_PATH
import torch

kernel_name = "sparse_fusion_2_thor_model"

kernel_template_filename = str(BRT_KERNEL_TEMPLATE_PATH / (kernel_name + ".cu"))

kernel_template_source = open(kernel_template_filename, "r").read()
kernel_func = CUDACompiler.generate_kernel(
    keyword_dict=None, template=kernel_template_source
)
data = torch.ones((8, 64, 64), device="cuda")
weight = torch.ones((8, 64, 64), device="cuda")
outdata = torch.ones((8, 64, 64), device="cuda")
kernel_func(data, weight, outdata)
print(outdata.shape)

In [11]:
from brt.jit.compiler import CUDACompiler
from brt.common import BRT_KERNEL_TEMPLATE_PATH
import time
import torch

kernel_name = "sample"

kernel_template_filename = str(BRT_KERNEL_TEMPLATE_PATH / (kernel_name + ".cu"))

kernel_template_source = open(kernel_template_filename, "r").read()
# print(kernel_template_source)
kernel_func = CUDACompiler.generate_kernel(
    {"batch_num": 2, "num_samples": 2}, kernel_template_source,
)
data = torch.ones((8, 64, 64), device="cuda")
weight = torch.ones((8, 64, 64), device="cuda")
outdata = torch.ones((8, 64, 64), device="cuda")
start_stamp = time.time()
kernel_func(data, weight, outdata)
end_stamp = time.time()
print("first time: {:.3f}".format((end_stamp - start_stamp) * 1000))

start_stamp = time.time()
kernel_func(data, weight, outdata)
end_stamp = time.time()
print("second time: {:.3f}".format((end_stamp - start_stamp) * 1000))


first time: 466.115
second time: 0.065


In [ ]:
from brt.common import BRT_KERNEL_TEMPLATE_PATH
from brt.jit.generic import GenericFunction
from brt.jit.compiler import CUDACompiler
import torch
kernel_name = "sample"

kernel_template_filename = str(BRT_KERNEL_TEMPLATE_PATH / (kernel_name + ".cu"))

kernel_template_source = open(kernel_template_filename, "r").read()

# print(kernel_template_source)
generic_function = GenericFunction(kernel_template_source)

code = generic_function.get_code("global")
# processed_template_fname = str(
#     BRT_KERNEL_TEMPLATE_PATH / ("processed_" + kernel_name + ".cu")
# )
# with open(processed_template_fname, "w") as f:
#     f.write(code)
data = torch.ones((8, 64, 64), device="cuda")
weight = torch.ones((8, 64, 64), device="cuda")
outdata = torch.ones((8, 64, 64), device="cuda")

func = CUDACompiler.generate_kernel(None, code)
func(data, weight, outdata)

In [12]:

import torch
from brt.jit.block_fuse import BlockFuser
from brt.common import BRT_KERNEL_TEMPLATE_PATH, log
from brt.jit.generic import GenericFunction
from brt.jit.compiler import CUDACompiler

log.set_level("jit", "DEBUG")

kernel_name = "sample"

kernel_template_filename = str(BRT_KERNEL_TEMPLATE_PATH / (kernel_name + ".cu"))

kernel_template_source = open(kernel_template_filename, "r").read()

block_fuser = BlockFuser([kernel_template_source, kernel_template_source])

code = block_fuser.get_code()

processed_template_fname = str(
    BRT_KERNEL_TEMPLATE_PATH / ("processed_" + kernel_name + ".cu")
)
with open(processed_template_fname, "w") as f:
    f.write(code)

fused_matmul = CUDACompiler.generate_kernel(None, code)

data = torch.ones((8, 64, 64), device="cuda")
weight = torch.ones((8, 64, 64), device="cuda")
outdata = torch.ones((8, 64, 64), device="cuda")
fused_matmul(data, weight, outdata, data, weight, outdata)


DEBUG:brainstorm.jit:Fusing blocks from 0 to 255 for 0-th block
DEBUG:brainstorm.jit:Fusing blocks from 256 to 511 for 1-th block


Error: Canceled future for execute_request message before replies were done